In [1]:
# USAGE
# python train_mask_detector.py --dataset dataset

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os




INIT_LR = 1e-4#taux d'apprentissage initial
EPOCHS = 20#le nombre d'époques d'entraînement
BS = 32#taille du lot

#Récupération de tous les imagePaths de l'ensemble de données 
print("[INFO] loading images...")
imagePaths = list(paths.list_images("dataset"))
data = []#Initialisation des listes de données 
labels = []#et d'étiquettes

#charger les chemins des images dataset et les labels with et without mask 
for imagePath in imagePaths:
	label = imagePath.split(os.path.sep) #La méthode os.path.split () en Python est utilisée pour diviser le nom du chemin en une tertaire tête milieu et queue. 
	label = imagePath.split(os.path.sep)[-2]#ne prendre que with ou without mask     
	# le chargement et le prétraitement des images
	image = load_img(imagePath, target_size=(224, 224))#redimensionnement à 224x224
	image = img_to_array(image)#la conversion au format de matrice 

	image = preprocess_input(image)# la mise à l'échelle des intensités de pixels de l'image d'entrée à la plage [-1, 1] (via la fonction de commodité preprocess_input)
	# Ajouter chaque image prétraité à la variable data
	data.append(image)
	labels.append(label)
 

	# convertir au format numpy
data = np.array(data, dtype="float32")#S'assurer que nos données d'entraînement sont au format de tableau NumPy
labels = np.array(labels)# une matrice contenant le label des images with or without mask 

lb = LabelBinarizer()
labels = lb.fit_transform(labels)#transformer en binaire(0.1) 0 without mask/1 with mask
labels = to_categorical(labels)#2catégories à gauche c'est sans masque et à droite avec masque(avec 2 dimensions) 

#print(lb.classes_)les 2 classes with et withou mask
# répartir nos données entre 20% de tests et 80% d'entrainement
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.2, stratify=labels, random_state=42)
#augmentation des données  en faisant des modifications sur les images du dataset comme la rotation le zoom ect.
aug = ImageDataGenerator(
	rotation_range=20,#rotation
	zoom_range=0.15,#zoom
	width_shift_range=0.2,#contrôle la largeur du réseau. 
	height_shift_range=0.2,#contrôle la hauteur du réseau.
	shear_range=0.15,#cisaillement
	horizontal_flip=True,
	fill_mode="nearest")
#Ceci est connu comme le multiplicateur de largeur dans le document MobileNet. - Si alpha <1,0, diminue proportionnellement le nombre de filtres dans chaque couche. - Si alpha> 1.0, augmente proportionnellement le nombre de filtres dans chaque couche. - Si alpha = 1, le nombre par défaut de filtres du papier est utilisé à chaque couche.
# charger MobileNetV2 : Un réseaux de neuronnes de classifications images et image_net c'est un ensemble d'images 
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))#'input est de taille 224x224

#construire la tête du modèle qui sera placé au dessus du modèle de base
headModel = baseModel.output# the output ou la sortie du model 
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)# réduction de la taille de la taille des donnéesen calculant la moyenne des valeurs de pixels des carrés 7x7
headModel = Flatten(name="flatten")(headModel)#applatir la matrice des valeurs en un seul vecteur
headModel = Dense(128, activation="relu")(headModel)#condenser ces matrices comme suit 128 fonctions qui en les combinant, et si on génère tous les pixels du viasage  génère une valeur 
#la fonction d'activation qui renvoie une valeur supérieure à 0
headModel = Dropout(0.5)(headModel)#supprimer the 'noises' (les bruits )#chaque détection est supprimée avec 1-la probabilité entre parenthèse
headModel = Dense(2, activation="softmax")(headModel)#nombre de types de visages différents(with or without mask)
#la fonction d'activation softmax met la plus grande probabilité des 2 cases 1 et l'autre à 0
#Le réglage fin est une stratégie que je recommande presque toujours pour établir un modèle de référence tout en gagnant un temps considérable.
# place the head FC model on top of the base model (this will become

model = Model(inputs=baseModel.input, outputs=headModel)

# arrêter la mise à jour du modèle de base alors qu'on fait toujours des mises à jour de la couche supérieure ou headmodel.
for layer in baseModel.layers:
	layer.trainable = False

# compiler notre model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)#compiler le modèle avec l'optimiseur adam
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics="accuracy")
#ce compilateur prend 2 valeurs l'optimiseur et l'erreur
#si l'alogorithme prédit une valeur alors la fonction d'erreur nous donne la médiocrité du modèle grâce à la fonction d'érreure
#l'algorithme nous utilisera la fonction d'optimisation pour donner une autre estimation
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=20)

#il repètra ca 20 fois selon le nombre de d'epochs(faire une prévision, calculer sa précision, utiliser l'optimisation pour calculer la prévision ect)
#prédiction du modèle
print("[INFO] evaluating network...")

#prédictions pour de nouvelles images
predIdxs = model.predict(testX, batch_size=BS)

# trouver l'index du label avec la plus grande probabilité
predIdxs = np.argmax(predIdxs, axis=1)


print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))
#enregistrer le modèle
print("[INFO] saving mask detector model...")
model.save("mask_detector.model", save_format="h5")
#Training loss c'est l'erreur calculée durant l'entrainement du model. Validation loss c'est l'erreur calculée epès avoir exécuté le jeu de donnée de validation "data_validation" via le réseau. 

# représentation graphiques de la précision et des erreurs  du modèle 
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")




[INFO] loading images...
[INFO] compiling model...
[INFO] training head...
Epoch 1/20


KeyboardInterrupt: 